## Fiducial tracking

In this example we'll discuss how to do fiducial tracking for drift correction.

### Configuration

In real experiments you'd have fiducials that you were also imaging, either in the same color channel as the data, or in a different color channel. You would then create two localization files, one that at least mostly contains the locations of the fiducials, and another file that contains all the localizations.

In [ ]:
import os
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

import numpy
numpy.random.seed(1) # Set seed so that the example is repeatable.

Generate sample data for this example.

In [ ]:
import storm_analysis.jupyter_examples.fiducial_tracking as fiducial_tracking

# Make a simulated movie.
fiducial_tracking.makeSampleData()

# Make a 3D-DAOSTORM analysis XML file.
fiducial_tracking.daoSTORMXML()


### Analyze fiducials with 3D-DAOSTORM

In [ ]:
import storm_analysis.daostorm_3d.mufit_analysis as mfit

if os.path.exists("fiducials.hdf5"):
    os.remove("fiducials.hdf5")
    
mfit.analyze("fiducials.tif", "fiducials.hdf5", "daostorm.xml")

### Track fiducials

In [ ]:
import storm_analysis.sa_utilities.fiducials as fiducials

# max_gap - Number of after which a fiducial will be considered to be lost.
# radius - Matching radius in pixels.
# reference_frame - Localizations from this frame will all be treated as fiducials.
#
fiducials.trackFiducials("fiducials.hdf5", max_gap = 4, radius = 1, reference_frame = 4)

### Plot results

In [ ]:
import matplotlib.pyplot as pyplot

with fiducials.SAH5Fiducials("fiducials.hdf5") as h5:
    fdcl = h5.getFiducial(1, fields = ["x", "y"])
    
    x = fdcl["x"] - numpy.mean(fdcl["x"])
    y = fdcl["y"] - numpy.mean(fdcl["y"])
    
    pyplot.plot(fdcl["frame"], x)
    pyplot.plot(fdcl["frame"], y)
    pyplot.show()
    

### Measure the drift

We'll do this by averaging together all of the fiducials.

In [ ]:
import scipy
import scipy.interpolate

x_drift = None
y_drift = None

with fiducials.SAH5Fiducials("fiducials.hdf5") as h5:
    ml = h5.getMovieLength()
    x = numpy.arange(ml)
    x_drift = numpy.zeros(ml)
    y_drift = numpy.zeros(ml)
    
    n_fiducials = 0
    for fdcl in h5.fiducialsIterator(fields = ["x", "y"]):

        # Verify this fiducial was present in most of the frames.
        if (fdcl["frame"].size < int(0.9 * ml)):
            continue
        
        # Use linear interpolation to fill in missing values.
        xpos_int = scipy.interpolate.interp1d(fdcl["frame"], 
                                              fdcl["x"], 
                                              kind = 'linear', 
                                              fill_value = 'extrapolate')
        
        ypos_int = scipy.interpolate.interp1d(fdcl["frame"], 
                                              fdcl["y"], 
                                              kind = 'linear', 
                                              fill_value = 'extrapolate')

        x_drift += xpos_int(x)
        y_drift += ypos_int(x)
        n_fiducials += 1
        
    print(n_fiducials, "fiducials out of", h5.getNFiducials(), "were used.")
    x_drift = x_drift/(float(n_fiducials))
    y_drift = y_drift/(float(n_fiducials))        

Since this a pretty common operation, the SAH5Fiducials class also provides this as a method.

In [ ]:
with fiducials.SAH5Fiducials("fiducials.hdf5") as h5:
    [ave_fdcl, n_averaged] = h5.averageFiducials(fields = ["x", "y"], min_frac_occupancy = 0.9)
    print(n_averaged, "fiducials out of", h5.getNFiducials(), "were used.")

print()
print("x difference {0:.3f}".format(numpy.max(numpy.abs(ave_fdcl["x"] - x_drift))))
print("y difference {0:.3f}".format(numpy.max(numpy.abs(ave_fdcl["y"] - y_drift))))


### Compare with ground truth

In [ ]:
gt_drift = numpy.loadtxt("drift.txt")

gt_x = gt_drift[:,0] - gt_drift[0,0]
x_drift -= x_drift[0]
pyplot.plot(gt_x, color = "black")
pyplot.scatter(numpy.arange(x_drift.size), x_drift, s = 10)
pyplot.show()
print("Mean x error: {0:3f} pixels".format(numpy.mean(numpy.abs(gt_x - x_drift))))

gt_y = gt_drift[:,1] - gt_drift[0,1]
y_drift -= y_drift[0]
pyplot.plot(gt_y, color = "black")
pyplot.scatter(numpy.arange(y_drift.size), y_drift, s = 10)
pyplot.show()
print("Mean y error: {0:3f} pixels".format(numpy.mean(numpy.abs(gt_y - y_drift))))


### Apply drift correction and verify

Here we're just applying it to the fiducial data file. Normally you'd probably apply it to the localizations data file.

In [ ]:
import storm_analysis.sa_library.sa_h5py as saH5Py

with saH5Py.SAH5Py("fiducials.hdf5") as h5:
    for i in range(x_drift.size):
        h5.setDriftCorrection(i, dx = -x_drift[i], dy = -y_drift[i])

In [ ]:
with fiducials.SAH5Fiducials("fiducials.hdf5") as h5:
    fdcl = h5.getFiducial(1, fields = ["x"])
    print("Uncorrected: {0:.3f} pixels".format(numpy.std(fdcl["x"])))
    
    fdcl = h5.getFiducial(1, drift_corrected = True, fields = ["x"])
    print("Corrected: {0:.3f} pixels".format(numpy.std(fdcl["x"])))